#### 1. Importing Required libraries

In [35]:
from dotenv import load_dotenv

load_dotenv()
import json
import os

from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatVertexAI
from langchain.embeddings import VertexAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import DeepLake

#### 2. Loading the Hyd data

In [36]:
loader = TextLoader('data/articles_content.txt')

In [37]:
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=20
)

docs = text_splitter.split_documents(documents)

In [38]:
embeddings = VertexAIEmbeddings()
dataset_path="hub://travana_db/hyd_embedding"

db = DeepLake(embedding=embeddings, dataset_path=dataset_path, overwrite=True)
db.add_documents(docs)

Your Deep Lake dataset has been successfully created!


Creating embedding data: 100%|██████████| 8/8 [00:03<00:00,  2.05it/s]
\

Dataset(path='hub://travana_db/hyd_embedding', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (79, 768)  float32   None   
    id        text      (79, 1)     str     None   
 metadata     json      (79, 1)     str     None   
   text       text      (79, 1)     str     None   


['228c9198-759f-11ee-b367-04421acf6dbe',
 '228c9199-759f-11ee-a2c0-04421acf6dbe',
 '228c919a-759f-11ee-9cab-04421acf6dbe',
 '228c919b-759f-11ee-85d6-04421acf6dbe',
 '228c919c-759f-11ee-8526-04421acf6dbe',
 '228c919d-759f-11ee-bf90-04421acf6dbe',
 '228c919e-759f-11ee-947b-04421acf6dbe',
 '228c919f-759f-11ee-b801-04421acf6dbe',
 '228c91a0-759f-11ee-a082-04421acf6dbe',
 '228c91a1-759f-11ee-bfde-04421acf6dbe',
 '228c91a2-759f-11ee-bbb3-04421acf6dbe',
 '228c91a3-759f-11ee-9f50-04421acf6dbe',
 '228c91a4-759f-11ee-86fa-04421acf6dbe',
 '228c91a5-759f-11ee-934a-04421acf6dbe',
 '228c91a6-759f-11ee-a347-04421acf6dbe',
 '228c91a7-759f-11ee-92a1-04421acf6dbe',
 '228c91a8-759f-11ee-ad38-04421acf6dbe',
 '228c91a9-759f-11ee-87fc-04421acf6dbe',
 '228c91aa-759f-11ee-a309-04421acf6dbe',
 '228c91ab-759f-11ee-954b-04421acf6dbe',
 '228c91ac-759f-11ee-b420-04421acf6dbe',
 '228c91ad-759f-11ee-bb06-04421acf6dbe',
 '228c91ae-759f-11ee-a294-04421acf6dbe',
 '228c91af-759f-11ee-8d01-04421acf6dbe',
 '228c91b0-759f-

In [8]:
db_loaded = DeepLake(
    dataset_path=dataset_path, 
    embedding=embeddings, 
    token=os.getenv("ACTIVELOOP_TOKEN"),
    read_only=True
)

Using embedding function is deprecated and will be removed in the future. Please use embedding instead.


Deep Lake Dataset in hub://travana_db/hyd_embedding already exists, loading from the storage


In [9]:
retriever = db_loaded.as_retriever()

In [10]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.
You can assume the question about the most recent state of the union address.

Chat History:
{chat_history}

Follow Up Input: {question}

Standalone question:"""

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [17]:
template = """
            You are an expert intelligent and creative AI itinerary planner with extensive knowledge of places worldwide. Your goal is to plan an optimized itinerary for the user based on their specific interests and preferences, geographical proximity, and efficient routes to minimize travel time. To achieve this, follow these instructions:

            1. Suggest three activities along with two restaurant per day. Each activity should include the name of the place, a brief description, estimated cost, and time to visit the place.

            2. Generate a well-structured itinerary including day-to-day activities, timings to visit each location, and estimated costs for the user's reference.

            3. Take into account factors such as geographical proximity between destinations, transportation options, and other logistical considerations when planning the route.
            
            By following these guidelines, you will create a comprehensive and optimized itinerary that meets the user's expectations while ensuring minimal travel time.
          
            Use this context to generate the itinerary
            
            ==========
            {context}
            ==========

            Question: {question}

            Structure the itinerary as follows:
            {{"Name":"name of the trip", "description":"description of the entire trip", "budget":"budget of the entire thing", "data": [{{"day":1, "day_description":"Description based on the entire day's places. in a couple of words, for example: `Urban Exploration`, `Historical Exploration`, `Spiritual Tour`, `Adventurous Journey`, `Dayout in a beach`, `Wildlife Safari`, `Artistic Getaway`, `Romantic Getaway`, `Desert Safari`, `Island Hopping Adventure`...",  "places":[{{"name":"Place Name", "description":"Place Description in two lines","time_to_visit": "time to visit this place, for example: 9:00 to 10:00", "budget":"cost"}}, {{"name":"Place Name 2", "description":"Place Description 2 in two lines","time_to_visit": "time to visit this place, for example 10:30 - 13:00", "budget":"cost"}}]}}, {{"day":2, "day_description": "Description based on the entire day's places", "places":[{{"name":"Place Name","description":"Place Description in two lines","time_to_visit": "time to visit this place","budget":"cost"}}, {{"name":"Place Name 2", "description":"Place Description 2 in two lines","time_to_visit": "time to visit this place","budget":"cost"}}]}}]}}
            Note: Do not include any extra information outside this structure.

            Answer in JSON format:"""

In [18]:
QA_PROMPT = PromptTemplate(template=template, input_variables=[
                           "context", "question"])

In [31]:
llm = ChatVertexAI(model_name='chat-bison', temperature=0.7, max_output_tokens=2004)
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_docs=True
)

model = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    get_chat_history=lambda h: h,
    condense_question_prompt=CONDENSE_QUESTION_PROMPT,
    combine_docs_chain_kwargs={"prompt": QA_PROMPT}
)

In [33]:
default_question = "Generate 3 day itinerary for a trip to Hyderabad, India. Consider budget, timings and requirements. Include estimated cost and timings to visit for each activity and restaurant"
#query = Prompt.ask("Your Question: ",  default=default_question)
result = model({"question": default_question})
print(result['answer'])

 ```json
{
  "Name": "Hyderabad Trip",
  "description": "A 3-day itinerary for exploring the city of Hyderabad, India.",
  "budget": "$500",
  "data": [
    {
      "day": 1,
      "day_description": "Historical Exploration",
      "places": [
        {
          "name": "Charminar",
          "description": "A 16th-century mosque and monument located in the old city of Hyderabad.",
          "time_to_visit": "9:00 to 10:00",
          "budget": "$5"
        },
        {
          "name": "Golconda Fort",
          "description": "A 16th-century fort located on a hilltop in Hyderabad.",
          "time_to_visit": "10:30 - 13:00",
          "budget": "$10"
        },
        {
          "name": "Qutb Shahi Tombs",
          "description": "A complex of tombs of the Qutb Shahi dynasty located in Hyderabad.",
          "time_to_visit": "13:30 - 15:00",
          "budget": "$5"
        }
      ]
    },
    {
      "day": 2,
      "day_description": "Cultural Exploration",
      "places": [

In [34]:
result = model({"question": "Change day 3 itinerary only to include restaurants"})
print(result['answer'])

 {"Name": "Hyderabad Trip", "description": "A trip to explore the beautiful city of Hyderabad.", "budget": "20000", "data": [{"day": 1, "day_description": "Historical Exploration", "places": [{"name": "Charminar", "description": "A 16th-century mosque and monument located in Hyderabad, India.", "time_to_visit": "9:00 to 10:30", "budget": "100"}, {"name": "Taramati Baradari", "description": "A 17th-century palace located in Hyderabad, India.", "time_to_visit": "11:00 to 12:30", "budget": "100"}, {"name": "Golconda Fort", "description": "A 16th-century fort located in Hyderabad, India.", "time_to_visit": "13:00 to 14:30", "budget": "100"}, {"name": "Chowmahalla Palace", "description": "A 19th-century palace located in Hyderabad, India.", "time_to_visit": "15:00 to 16:30", "budget": "100"}, {"name": "Salar Jung Museum", "description": "A museum located in Hyderabad, India.", "time_to_visit": "17:00 to 18:30", "budget": "100"}, {"name": "Birla Mandir", "description": "A Hindu temple locate